## Question 3 - Train a Pure CNN with less than 10000 trainable parameters using the MNIST Dataset having minimum validation accuracy of 99.40%
**Note -**
1. Code comments should be given for proper code understanding.
2. Implement in both PyTorch and Tensorflow respectively


In [1]:
# PyTorch Implementation
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Hyperparameters
num_epochs = 10
batch_size = 128
learning_rate = 0.001

In [4]:
# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform, download=True
)

100%|██████████| 9912422/9912422 [00:00<00:00, 491682765.02it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 47747612.86it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 186216413.92it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20684613.21it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

In [7]:
# Define the CNN model
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = nn.functional.relu(self.conv1(x))
        out = nn.functional.max_pool2d(out, 2)
        out = nn.functional.relu(self.conv2(out))
        out = nn.functional.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = nn.functional.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [8]:
# Instantiate the model
model = PureCNN().to(device)

In [9]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}")

Epoch [1/10], Step [100/469], Loss: 0.2846
Epoch [1/10], Step [200/469], Loss: 0.1544
Epoch [1/10], Step [300/469], Loss: 0.0959
Epoch [1/10], Step [400/469], Loss: 0.0686
Epoch [2/10], Step [100/469], Loss: 0.0595
Epoch [2/10], Step [200/469], Loss: 0.0508
Epoch [2/10], Step [300/469], Loss: 0.0350
Epoch [2/10], Step [400/469], Loss: 0.0882
Epoch [3/10], Step [100/469], Loss: 0.0523
Epoch [3/10], Step [200/469], Loss: 0.0410
Epoch [3/10], Step [300/469], Loss: 0.0968
Epoch [3/10], Step [400/469], Loss: 0.0266
Epoch [4/10], Step [100/469], Loss: 0.0172
Epoch [4/10], Step [200/469], Loss: 0.0190
Epoch [4/10], Step [300/469], Loss: 0.0334
Epoch [4/10], Step [400/469], Loss: 0.0829
Epoch [5/10], Step [100/469], Loss: 0.0528
Epoch [5/10], Step [200/469], Loss: 0.0720
Epoch [5/10], Step [300/469], Loss: 0.0052
Epoch [5/10], Step [400/469], Loss: 0.0083
Epoch [6/10], Step [100/469], Loss: 0.0256
Epoch [6/10], Step [200/469], Loss: 0.0061
Epoch [6/10], Step [300/469], Loss: 0.0142
Epoch [6/10

In [11]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 99.00%


## Tensorflow Implementation

In [12]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [13]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [14]:
# Normalize pixel values to range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

In [15]:
# Reshape images to 4D tensor
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [16]:
# Define CNN architecture
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(10, activation='softmax')
])

In [17]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
#Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 10)                8

In [19]:
# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 5s 5ms/step - loss: 0.3264 - accuracy: 0.9126 - val_loss: 0.0909 - val_accuracy: 0.9738
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0887 - accuracy: 0.9736 - val_loss: 0.0724 - val_accuracy: 0.9787
Epoch 3/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0682 - accuracy: 0.9790 - val_loss: 0.0603 - val_accuracy: 0.9818
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0572 - accuracy: 0.9827 - val_loss: 0.0447 - val_accuracy: 0.9851
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0501 - accuracy: 0.9845 - val_loss: 0.0482 - val_accuracy: 0.9848
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0453 - accuracy: 0.9860 - val_loss: 0.0436 - val_accuracy: 0.9856
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0422 - accuracy: 0.9870 - val_loss: 0.0417 - val_accuracy: 0.9866
Epoch 

In [20]:
# Evaluate accuracy on test set
_, accuracy = model.evaluate(x_test, y_test)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 2ms/step - loss: 0.0429 - accuracy: 0.9877
Validation Accuracy: 98.77%
